In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        super().__init__()
        self.features = torch.tensor(dataframe.iloc[:, :-1].values, dtype=torch.float32)
        self.labels = torch.tensor(dataframe.iloc[:, -1].values, dtype=torch.long)
    
    def __len__(self):
        return self.features.size(0)
    
    def __getitem__(self, index):
        pass

In [ ]:
# Loading the dataset and encoding it to numerical values
dataset = pd.read_excel('./dataset/Dry_Bean_Dataset.xlsx')
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Creating the custom dataset
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)

# Creating the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [20]:
# Making Our Neural Network
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.first_layer = nn.Linear(16, 32)
        self.second_layer = nn.Linear(32, 64)
        self.third_layer = nn.Linear(64, 128)
        self.fourth_layer = nn.Linear(128, 64)
        self.fifth_layer = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 7)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.first_layer(x))
        x = self.relu(self.second_layer(x))
        x = self.relu(self.third_layer(x))
        x = self.relu(self.fourth_layer(x))
        x = self.relu(self.fifth_layer(x))
        x = self.output_layer(x)
        return x

In [ ]:
# Defining the train and Test functions
def train_model(model, train_loader, optimiser, criterion, epochs):
    for epoch in epochs:
        for batch_index, (data, labels) in enumerate(train_loader):
            optimiser.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            optimiser.step()
            if batch_index % 10 == 0:
                print(f"Epoch: {epoch} Batch: {batch_index} Loss: {loss.item()}")

def test_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy of the model on the test set is: {100 * correct / total}")

In [ ]:
if __name__ == "__main__":
    model = MLP()
    optimiser = optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    epochs = 100
    train_model(model, train_dataloader, optimiser, criterion, epochs)
    test_model(model, test_dataloader)